In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

In [2]:
# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
# Create aiport dataframe
cities_clean_df = pd.read_csv('Cities_Output2.csv')
cities_clean_df.dropna(inplace = True) 
cities_clean_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloud Coverage,Wind Speed,Country,Date
0,0,lubao,14.92,120.55,91.40,59.0,75.0,4.70,PH,1593842888
1,1,yellowknife,62.46,-114.35,57.99,100.0,90.0,11.41,CA,1593842315
2,2,katsuura,35.13,140.30,82.40,69.0,75.0,28.86,JP,1593842890
3,3,vostok,46.49,135.88,78.62,44.0,41.0,7.00,RU,1593842890
4,4,pisco,-13.70,-76.22,57.00,93.0,20.0,4.70,PE,1593842891
...,...,...,...,...,...,...,...,...,...,...
547,588,igarka,67.47,86.58,74.35,44.0,99.0,20.71,RU,1593843250
548,589,lodwar,3.12,35.60,80.06,48.0,49.0,10.94,KE,1593843250
549,590,san quintin,30.48,-115.95,66.16,84.0,0.0,10.42,MX,1593843251
550,591,bardiyah,31.76,25.09,77.07,68.0,0.0,9.04,LY,1593843252


In [4]:
locations = cities_clean_df[["Lat", "Lng"]]
humidity = cities_clean_df["Humidity"].astype(float)


In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
ideal_weather =cities_clean_df.loc[(cities_clean_df["Max Temp"] > 70) & (
    cities_clean_df["Max Temp"] < 80) & (
    cities_clean_df["Wind Speed"] < 10) & (
    cities_clean_df["Cloud Coverage"] ==0), :]

In [7]:
ideal_weather=pd.DataFrame(ideal_weather)
hotel=ideal_weather[["City", "Lat", "Lng", "Country"]]
hotel_df=pd.DataFrame(hotel)

#save cities only, build a new database


hotel_df["Hotel Name"]=''

#making sure maximum 10 rows
hotel_df=hotel_df.head(10)

hotel_df

,City,Lat,Lng,Country,Hotel Name
221,sarkikaraagac,38.08,31.37,TR,
258,misratah,32.38,15.09,LY,
293,waddan,29.16,16.14,LY,
352,gladstone,-23.85,151.25,AU,
375,presidente medici,-11.18,-61.90,BR,
383,obidos,-1.92,-55.52,BR,
422,uray,60.12,64.78,RU,
431,maridi,4.92,29.47,SS,
469,la orilla,17.98,-102.23,MX,
534,sukhoy log,56.91,62.03,RU,


In [8]:
#Need the google geocode coordinates in order to ge the APis
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel.iterrows():
    params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key,
    "keyword":"hotel"
} 
    
    
    
    lat=str(row["Lat"])
    lng=str(row["Lng"])
    
    
    combined= lat + ',' ' ' + lng
    
    
    params['location'] = combined

    
    response = requests.get(base_url, params=params).json()
       
    try:
        hotel_df.loc[index, "Hotel Name"] =(response['results'][0]['name'])

    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] =("No hotel within 5000m of coordinates")
   
            

    
   

In [9]:
hotel_df.dropna(inplace = True) 


In [10]:

 # Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [11]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [12]:
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…